<div style="border-radius: 10px; background-color: #A478B8">
    <h1 style="color: white; padding: 1rem">Introduction</h1>
</div>

This is a beginner-friendly notebook that attempts to perform **Exploratory Data Analysis** on the **[Pima Indians Diabetes Dataset](https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database)** and eventually train a Machine Learning model on it and enhance the predictions by fine-tuning the model.

In [ ]:
# for data wrangling
import numpy as np
import pandas as pd

# for data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# modelling
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.linear_model import SGDClassifier

from sklearn.model_selection import cross_val_predict, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report, PrecisionRecallDisplay, RocCurveDisplay, accuracy_score, confusion_matrix, ConfusionMatrixDisplay

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

<div style="border-radius: 10px; background-color: #A478B8">
    <h1 style="color: white; padding: 1rem">Know the Data</h1>
</div>

Let us first look at what information the dataset contains. One thing to note is, all patients here are females **at least 21 years old** of Pima Indian heritage.

The dataset has the following **features** (columns):

- **<span style="color: darkblue">Pregnancies</span>**: Number of times pregnant
- **<span style="color: darkblue">Glucose</span>**: Plasma glucose concentration, 2 hours in an oral glucose tolerance test
- **<span style="color: darkblue">BloodPressure</span>**: Diastolic blood pressure ($mm \cdot Hg$)
- **<span style="color: darkblue">SkinThickness</span>**: Triceps skin fold thickness ($mm$)
- **<span style="color: darkblue">Insulin</span>**: 2-Hour serum insulin ($mu \cdot \dfrac{U} {ml}$)
- **<span style="color: darkblue">BMI</span>**: Body mass index (weight in $kg$ / height in $m^2$)
- **<span style="color: darkblue">DiabetesPedigreeFunction</span>**: Diabetes pedigree function (indicates the function which scores likelihood of diabetes based on family history)
- **<span style="color: darkblue">Age</span>**: Age (in years)
- **<span style="color: darkblue">Outcome</span>**: Whether patient is diagnosed with Diabetes (0 for No, 1 for Yes)

**<span style="color: purple"> Outcome is our target variable </span>**.

<div style="border-radius: 10px; background-color: #A478B8">
    <h1 style="color: white; padding: 1rem">Load the dataset</h1>
</div>

Using **Pandas** we load the dataset.

In [ ]:
df = pd.read_csv("/kaggle/input/pima-indians-diabetes-database/diabetes.csv")
df

An important thing to note here is that:

> This Pima Indians Diabetes dataset contains only **768 rows** and **9 features**. If we want to split the dataset into training and testing set, we need to ensure that the test set is *representative* of the whole dataset. This helps in avoiding the **Sampling Bias** when the training and testing sets are chosen randomly.

In [ ]:
df.info()

We clearly see that apart from the feature `Outcome`, every other feature is numerical and continuous in nature.

In [ ]:
df.describe()

**<span style="color: purple">Note: The minimum value of the features `Glucose`, `BloodPressure`, `SkinThickness`, `Insulin` and `BMI` is 0. This is logically incorrect as these values cannot be 0. Thus, this can be safely called "missing data" in our case. We need to either drop the 0-valued rows or we need to replace them with the *mean* or *median* value of that feature.</span>**

Let's check for null values (if any):

In [ ]:
# helper function
def count_na(df, col):
    print(f"Null values in {col}: ", df[col].isna().sum())

for feat in df.columns:
    count_na(df, feat)

Good! There are no null values in our dataset. Let us move on to *visualizing* the dataset to gather more insights about the data.

<div style="border-radius: 10px; background-color: #A478B8">
    <h1 style="color: white; padding: 1rem">Data visualization</h1>
</div>

In this section, we will start visualizing the features of the dataset one by one. Firstly, **Univariate** feature visualization will be done, then we will move onto **Multivariate** feature visualization.

> To learn more about what **graphs** are useful for what **data-types**, check out this notebook here: [Statistical Data Types and Graphs (using Seaborn)](https://www.kaggle.com/code/maharshipandya/statistical-data-types-and-graphs-using-seaborn)

In [ ]:
# Setting some styles
sns.set_style("darkgrid")
sns.set_palette("viridis")

<h1 style="color: darkblue">Univariate Analysis</h1>
<hr/>

### Analysis of Pregnancies

As observed, `Pregnancies` is a **Quantitative** feature. There are many plots to analyse these type of data. Histograms, Box plots and Violin plots, are useful to know how the data is distributed.

In [ ]:
fig1, ax1 = plt.subplots(1, 2, figsize=(20, 7))
fig2, ax2 = plt.subplots(figsize=(20, 7))

sns.histplot(data=df, x="Pregnancies", kde=True, ax=ax1[0])
sns.boxplot(data=df, x="Pregnancies", ax=ax1[1])

sns.violinplot(data=df, x="Pregnancies", ax=ax2)

plt.show()

In [ ]:
print("Median of Pregnancies: ", df["Pregnancies"].median())
print("Maximum of Pregnancies: ", df["Pregnancies"].max())

In [ ]:
df["Pregnancies"].value_counts()

From the above analysis we observe that:

- Most patients had 0, 1 or 2 pregnancies.
- Median value of `Pregnancies` is **3**.
- Also, patients had upto **17** pregnancies!

There are 3 outliers on the boxplot. But, let's not remove them for now.

### Analysis of Outcome (Target Variable)

A Count plot and a Pie chart will be two useful plots to analyse the `Outcome` column as it is a categorical feature. Usefulness in the sense, both the plots will allow us to observe the distribution of each category in the feature.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 7))

sns.countplot(data=df, x="Outcome", ax=ax[0])
df["Outcome"].value_counts().plot.pie(explode=[0.1, 0], autopct="%1.1f%%", labels=["No", "Yes"], shadow=True, ax=ax[1])

plt.show()

We observe from the above plot that:

- **65.1% patients in the dataset do NOT have diabetes.**
- **34.9% patients in the dataset has diabetes.**

### Analysis of Glucose

`Glucose` is a **Quantitative** feature. Histograms, Box plots and Violin plots, are useful to know how the data is distributed.

In [ ]:
fig3, ax3 = plt.subplots(1, 2, figsize=(20, 7))
fig4, ax4 = plt.subplots(figsize=(20, 7))

sns.histplot(data=df, x="Glucose", kde=True, ax=ax3[0])
sns.boxplot(data=df, x="Glucose", ax=ax3[1])

sns.violinplot(data=df, x="Glucose", ax=ax4)

plt.show()

In [ ]:
print("Median of Glucose: ", df["Glucose"].median())
print("Maximum of Glucose: ", df["Glucose"].max())
print("Mean of Glucose: ", df["Glucose"].mean())

In [ ]:
print("Rows with Glucose value of 0: ", df[df["Glucose"] == 0].shape[0])

We observe that:

- Median (117.0) and mean (120.8) of `Glucose` lie very close to each other i.e. the distribution is more or less **symmetric and uniform**.
- As seen from the box plot, an outlier lies on 0-value, which I talked about earlier.
- There are **5 rows** with `Glucose` value as 0. This is not logical, so we need to keep this in mind.

### Analysis of Blood Pressure

`BloodPressure` is a **Quantitative** feature. Histograms, Box plots and Violin plots, are useful to know how the data is distributed.

In [ ]:
fig5, ax5 = plt.subplots(1, 2, figsize=(20, 7))
fig6, ax6 = plt.subplots(figsize=(20, 7))

sns.histplot(data=df, x="BloodPressure", kde=True, ax=ax5[0])
sns.boxplot(data=df, x="BloodPressure", ax=ax5[1])

sns.violinplot(data=df, x="BloodPressure", ax=ax6)

plt.show()

In [ ]:
print("Median of Blood Pressure: ", df["BloodPressure"].median())
print("Maximum of Blood Pressure: ", df["BloodPressure"].max())
print("Mean of Pressure: ", df["BloodPressure"].mean())

In [ ]:
print("Rows with BloodPressure value of 0: ", df[df["BloodPressure"] == 0].shape[0])

We observe that:
​
- Median (72.0) and mean (69.1) of `BloodPressure` lie very close to each other i.e. the distribution is more or less **symmetric and uniform**.
- As seen from the box plot and violin plot, some outliers lie on 0-value, which I talked about earlier.
- There are **35 rows** with `BloodPressure` value as 0. This is not logical.

### Analysis of Insulin

Plotting Histogram, Box plot and Violin plot for `Insulin`.

In [ ]:
fig7, ax7 = plt.subplots(1, 2, figsize=(20, 7))
fig8, ax8 = plt.subplots(figsize=(20, 7))

sns.histplot(data=df, x="Insulin", kde=True, ax=ax7[0])
sns.boxplot(data=df, x="Insulin", ax=ax7[1])

sns.violinplot(data=df, x="Insulin", ax=ax8)

plt.show()

In [ ]:
print("Rows with Insulin value of 0: ", df[df["Insulin"] == 0].shape[0])

The plots for `Insulin` are highly skewed. Also, the 0-value logical error is the most for this feature. **374 out of 768** instances have value of `Insulin` as 0.

### Analysis of BMI

Plotting Histogram, Box plot and Violin plot for `BMI`.

In [ ]:
fig9, ax9 = plt.subplots(1, 2, figsize=(20, 7))
fig10, ax10 = plt.subplots(figsize=(20, 7))

sns.histplot(data=df, x="BMI", kde=True, ax=ax9[0])
sns.boxplot(data=df, x="BMI", ax=ax9[1])

sns.violinplot(data=df, x="BMI", ax=ax10)

plt.show()

In [ ]:
print("Median of BMI: ", df["BMI"].median())
print("Maximum of BMI: ", df["BMI"].max())
print("Mean of BMI: ", df["BMI"].mean())

In [ ]:
print("Rows with BMI value of 0: ", df[df["BMI"] == 0].shape[0])

We observe that:

- Median (32.0) and Mean (31.9) of `BMI` are very close to each other. Thus, the distribution is more or less **symmetric and uniform**
- Maximum BMI is 67.1
- There are **11 rows** with `BMI` value as 0

### Analysis of Diabetes Pedigree Function

`DiabetesPedigreeFunction` is a **continuous and quantitative** variable.

In [ ]:
fig11, ax11 = plt.subplots(1, 2, figsize=(20, 7))
fig12, ax12 = plt.subplots(figsize=(20, 7))

sns.histplot(data=df, x="DiabetesPedigreeFunction", kde=True, ax=ax11[0])
sns.boxplot(data=df, x="DiabetesPedigreeFunction", ax=ax11[1])

sns.violinplot(data=df, x="DiabetesPedigreeFunction", ax=ax12)

plt.show()

In [ ]:
print("Median of DiabetesPedigreeFunction: ", df["DiabetesPedigreeFunction"].median())
print("Maximum of DiabetesPedigreeFunction: ", df["DiabetesPedigreeFunction"].max())
print("Mean of DiabetesPedigreeFunction: ", df["DiabetesPedigreeFunction"].mean())

We observe that:

- The histogram is higly skewed on the left side.
- There are many outliers in the Box plot.
- Violin plot distribution is dense in the interval `0.0 - 1.0`

### Analysis of Age

Plotting Histogram, Box plot and Violin plots for `Age`.

In [ ]:
fig13, ax13 = plt.subplots(1, 2, figsize=(20, 7))
fig14, ax14 = plt.subplots(figsize=(20, 7))

sns.histplot(data=df, x="Age", kde=True, ax=ax13[0])
sns.boxplot(data=df, x="Age", ax=ax13[1])

sns.violinplot(data=df, x="Age", ax=ax14)

plt.show()

In [ ]:
print("Median of Age: ", df["Age"].median())
print("Maximum of Age: ", df["Age"].max())
print("Mean of Age: ", df["Age"].mean())

We again observe that:

- The distribution of Age is skewed on the left side.
- There are some outliers in the Box plot for Age.

<h1 style="color: darkblue">Multivariate Analysis</h1>
<hr/>



### Analysis of Glucose and Outcome

Since `Glucose` is a continuous feature, we plot a histogram with its hue based on `Outcome`.

In [ ]:
fig15, ax15 = plt.subplots(figsize=(20, 8))

sns.histplot(data=df, x="Glucose", hue="Outcome", shrink=0.8, multiple="fill", kde=True, ax=ax15)
plt.show()

From the above plot, we see a **positive linear correlation**.

- As the value of `Glucose` increases, the count of patients having diabetes increases i.e. value of `Outcome` as 1, increases.
- Also, after the `Glucose` value of **125**, there is a steady increase in the number of patients having `Outcome` of 1.
- Note, when `Glucose` value is 0, it means the measurement is missing. We need to fill that values with the *mean* or *median* and then it will make sense.

So, there is a significant amount of *positive* linear correlation.

### Analysis of BloodPressure and Outcome

`BloodPressure` is continuous and `Outcome` is binary feature. So, plotting a histogram for `BloodPressure` with its hue based on `Outcome`.

In [ ]:
fig16, ax16 = plt.subplots(figsize=(20, 8))

sns.histplot(data=df, x="BloodPressure", hue="Outcome", shrink=0.8, multiple="dodge", kde=True, ax=ax16)
plt.show()

We observe that, `Outcome` and `BloodPressure` do **NOT** have a positive or negative linear correlation. The value of `Outcome` do not increase linearly as value of `BloodPressure` increases.

However, for `BloodPressure` values greater than 82, count of patients with `Outcome` as 1, is more.

### Analysis of BMI and Outcome

In [ ]:
fig17, ax17 = plt.subplots(figsize=(20, 8))

sns.histplot(data=df, x="BMI", hue="Outcome", shrink=0.8, multiple="fill", kde=True, ax=ax17)
plt.show()

From the above plot, a **positive linear correlation** is evident for `BMI`.

### Analysis of Age and Outcome

`Age` is continuous so plotting a histogram with hue based on `Outcome`.

In [ ]:
fig18, ax18 = plt.subplots(figsize=(20, 8))

sns.histplot(data=df, x="Age", hue="Outcome", shrink=0.8, multiple="dodge", kde=True, ax=ax18)
plt.show()

For `Age` greater than 35 years, the chances of patients having diabetes increases as evident from the plot i.e. The number of patients having diabetes is more than the number of people **NOT** having diabetes. But, it does not hold true for ages like **60+**, somehow.

There is *some* positive linear correlation though.

### Analysis of Pregnancies and Outcome

In [ ]:
fig19, ax19 = plt.subplots(figsize=(20, 8))

sns.histplot(data=df, x="Pregnancies", hue="Outcome", shrink=0.8, multiple="fill", kde=True, ax=ax19)
plt.show()

There is *some* positive linear correlation of `Pregnancies` with `Outcome`.

<h1 style="color: darkblue">Analyzing Correlations</h1>
<hr/>

Let us plot a **heatmap** of the correlation matrix of different features.

In [ ]:
# The 2D correlation matrix
corr_matrix = df.corr()

In [ ]:
# Plotting the heatmap of corr

fig20, ax20 = plt.subplots(figsize=(20, 7))
dataplot = sns.heatmap(data=corr_matrix, annot=True, ax=ax20)
plt.show()

In [ ]:
corr_matrix["Outcome"].sort_values(ascending=False)

We observe that:

- `Glucose` has the maximum positive linear correlation with `Outcome`, which is logical.
- `BloodPressure` has the lowest positive linear correlation with `Outcome`.
- No feature has a negative linear correlation with `Outcome`.

<div style="border-radius: 10px; background-color: #A478B8">
    <h1 style="color: white; padding: 1rem">Preparing the Data</h1>
</div>

It is now time to prepare the data for machine learning algorithms to train on. There are a few things we need to do.

- **Split into training and testing set**: Using **Stratified Split**, we split the whole dataset into training and testing set such that the testing set is representative of the entire dataset.
- **Fill in the missing values**: Some features have 0-values which is not logical. So we need to replace them with either *mean* or *median*.
- **Scaling the feature values**: We apply standard scaling on the feature values, so that the ranges of the features are not too varied.

# Split into Training and Testing set

Since `Glucose` has the highest positive linear correlation with `Outcome`, we will split the dataset based on the categories of `Glucose`. This is called **Stratified Splitting** with the *strata* being `Glucose` categories. The categories do not exist just yet. We need to create them.

Below code cell creates a new feature called `Glucose_cat` which divides the `Glucose` feature having range $ [0 - 199] $ into **5 categories**.
The categories being:

- $(-1, 40]$
- $(40, 80]$
- $(80, 120]$
- $(120, 160]$
- $(160, \infty]$

In [ ]:
newdf = df

In [ ]:
# segment the dataset into bins

newdf["Glucose_cat"] = pd.cut(newdf["Glucose"],
                           bins=[-1, 40, 80, 120, 160, np.inf],
                          labels=[1, 2, 3, 4, 5])

In [ ]:
newdf["Glucose_cat"].value_counts()

In [ ]:
fig21, ax21 = plt.subplots(figsize=(20, 7))

newdf["Glucose_cat"].hist(ax=ax21)
plt.show()

Using **Scikit-Learn's `Stratified Shuffle Split`** we can split the dataset into Training and Testing set.

In [ ]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=3301)

In [ ]:
for train_index, test_index in split.split(newdf, newdf["Glucose_cat"]):
    strat_train_set = newdf.loc[train_index]
    strat_test_set = newdf.loc[test_index]

We can now compare the proportions of various `Glucose` categories between the Testing set and the entire dataset.
The output below shows the various proportions.

In [ ]:
def get_glucose_proportions(ndf):
    print(ndf["Glucose_cat"].value_counts() / len(ndf))

print("Entire Dataset: ")
get_glucose_proportions(newdf)
print("\n")
print("-"*30)
print("\nTesting set: ")
get_glucose_proportions(strat_test_set)

We now drop the `Glucose_cat` column to bring the data back to its original form.

In [ ]:
for set_ in (strat_train_set, strat_test_set):
    set_.drop(columns=["Glucose_cat"], inplace=True)

# Fill in the missing values

We will replace the 0-values of the columns:

- `Glucose`
- `BloodPressure`
- `SkinThickness`
- `Insulin`
- `BMI`

with their median values.

However, we will store the medians in an array, so that the test set can be replaced by that medians.

> Note: It is essential to split the dataset before performing **Imputation** (replacement of missing values) and **Standardization** to avoid data leaking of the test set into the training set. We want our model to perform good on unseen data.

In [ ]:
# to store medians
meds = []
feats = ["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]

for feat in feats:
    meds.append(strat_train_set[feat].median())
    
print("Medians are: ", meds)

In [ ]:
# helper function
def replace_with_median(ndf, feat, value):
    ndf[feat] = ndf[feat].replace(0, value)
    
for i, feat in enumerate(feats):
    replace_with_median(strat_train_set, feat, meds[i])
    replace_with_median(strat_test_set, feat, meds[i])

With this, we replaced all the missing values with the median (of that column) in the Training set and used those *learned* medians to replace the missing values in the Testing set.

In [ ]:
X_train = strat_train_set.drop(columns="Outcome")
y_train = strat_train_set["Outcome"]

X_test = strat_test_set.drop(columns="Outcome")
y_test = strat_test_set["Outcome"]

# Scaling the features

Since the ranges of different features vary too much, it is necessary to *scale* them so that the Machine Learning models can perform even better. We will use **Scikit-Learn's Standard Scaler** to scale the features.

In [ ]:
stdscaler = StandardScaler()
stdscaler.fit(X_train)

In [ ]:
X_train_ = stdscaler.transform(X_train)
X_test_ = stdscaler.transform(X_test)

print("Scaled training set: ", X_train_)
print("Scaled testing set: ", X_test_)

Done! We now have our Training set `X_train_` (numpy array), Testing set `X_test_` (numpy array) and labels `y_train` and `y_test`, for **supervised learning**.

<div style="border-radius: 10px; background-color: #A478B8">
    <h1 style="color: white; padding: 1rem">Classification - Supervised Learning</h1>
</div>

Data preparation is done, now its time to run ML algorithms on the preprocessed data. Here, I have written a custom helper function to compare a list of Classifiers, using their **Classification Report**, **Confusion Matrix**, **Precision Recall Curve** and **Reciever Operating Characteristic Curve**.

In [ ]:
def comp_esti(esti):
    esti.fit(X_train_, y_train)
    esti_test_preds = esti.predict(X_test_)
    
    print(f"{esti} Accuracy score: ", accuracy_score(y_test, esti_test_preds))
    print(f"\n{esti} Classification report:\n", classification_report(y_test, esti_test_preds, digits=6))
    
    # confusion matrix
    cf_mat = confusion_matrix(y_test, esti_test_preds)
    fig, ax = plt.subplots(figsize=(10, 8))
    sns.heatmap(data=cf_mat, annot=True, ax=ax)
    plt.show()

Next, we provide a list of classifiers as a parameter to this function...

In [ ]:
estimators = [
    RandomForestClassifier(random_state=3301),
    SVC(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    LogisticRegression(),
    DecisionTreeClassifier(),
    KNeighborsClassifier()
]

for esti in estimators:
    comp_esti(esti)

In [ ]:
# Voting classifier
es1 = RandomForestClassifier(random_state=3301)
es2 = SVC(probability=True)
es3 = GradientBoostingClassifier()

esfinal = VotingClassifier(estimators=[
    ("rfc", es1), ("svc", es2), ("grb", es3)
], voting="soft")

comp_esti(esfinal)

> Note: Which model to choose? That highly depends on what your application is targeted to do. In essence, do you wanna go for high precision or high recall?

However, as an example let's fine-tune the **Random Forest Classifier**.

<div style="border-radius: 10px; background-color: #A478B8">
    <h1 style="color: white; padding: 1rem">Fine-tuning our Model</h1>
</div>

Using **Randomized Search CV**, let's try to find better hyperparameters for the **Random Forest Classifier**.

In [ ]:
# Parameters of random forest classifier
n_estimators = np.linspace(50, 300, int((300 - 50) / 20), dtype=int)
max_depth = [1, 5, 10, 50, 100, 200, 300]
min_samples_split = [2, 4, 6]
max_features = ["sqrt", "log2"]
bootstrap = [True, False]

distributions = {
    "n_estimators": n_estimators,
    "max_depth": max_depth,
    "min_samples_split": min_samples_split,
    "max_features": max_features,
    "bootstrap": bootstrap
}

In [ ]:
# Randomised search cv
from sklearn.model_selection import RandomizedSearchCV

rfc = RandomForestClassifier(random_state=3301)
random_search_cv = RandomizedSearchCV(
    rfc,
    param_distributions=distributions,
    n_iter=30,
    cv=5,
    n_jobs=4
)

search = random_search_cv.fit(X_train_, y_train)

The results of this Randomized Search are stored in a dictionary named `cv_results_`. Let us print these results just to get an idea of what parameters were tested by our Randomized Search.

In [ ]:
cvres = search.cv_results_

for score, params, rank in zip(cvres["mean_test_score"], cvres["params"], cvres["rank_test_score"]):
    print(score, params, rank)
    

Also, the best estimator out of these tested ones is stored in a variable called `best_estimator_`. We can use this estimator as our fine-tuned model.

In [ ]:
rfc_finetuned = search.best_estimator_
rfc_finetuned.fit(X_train_, y_train)

best_preds = rfc_finetuned.predict(X_test_)

fig, ax = plt.subplots(1, 2, figsize=(20, 10))
PrecisionRecallDisplay.from_predictions(y_test, best_preds, ax=ax[0])
RocCurveDisplay.from_predictions(y_test, best_preds, ax=ax[1])

print(classification_report(y_test, best_preds, digits=5))
plt.show()

As you can see, by fine-tuning the **Random Forest Classifier**:

- We increased the average precision from **0.68** to **0.70**.
- We increased the average recall from **0.67** to **0.69**.
- We increased the accuracy from **70%** to **72%**.

**<span style="font-size: 14px; color: darkblue">If this notebook helped you a slightest bit, do consider upvoting it and leaving a comment below! Feel free to extend this notebook with more knowledge. Thank you! ❤️</span>**